# Setup

In [6]:
import sys
sys.path.append("..")
import os
os.chdir("./..")
from utils import get_opensearch_client
os.chdir("./lab")

os_client = get_opensearch_client()

# Opensearch Indices

In [7]:
response = os_client.cat.indices(format='json')

for index in response:
    print(index['index'])


test-index
.opensearch-observability
.plugins-ml-config
security-auditlog-2024.03.17
.opendistro_security


# Get docs from a index

In [11]:
def get_all_docs(index_name):
    # Execute the search query to get the first 1000 documents
    response = os_client.search(
        index=index_name,
        body={"query": {"match_all": {}}},
        size=1000
    )

    # Iterate through the hits and print document details
    for doc in response['hits']['hits']:
        print(doc['_id'], doc['_source'])

# Example usage
index_name = 'test-index'
get_all_docs(index_name)

1 {'action': 'index', 'title': 'El barco velero', 'content': 'Content of document 1'}
2 {'action': 'index', 'title': 'Hawai', 'content': 'Content of document 2'}
3 {'action': 'index', 'title': 'Document 3 Updated', 'content': 'Updated content of document 3'}
4 {'action': 'index', 'title': 'Document 4 Updated', 'content': 'Updated content of document 3'}
